# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Predictions Infox - Comparaison Bag of Words

---

### Conseils JW
- XGBoost pour données structurées
- Spacy
- Fasttext avec api python : https://fasttext.cc/docs/en/python-module.html
- Comment fusionner les vecteurs de mots : Doc2Vec
- Réseau de neurones : faire petit, metrique AUC
- Word Embedings : https://fasttext.cc/docs/en/crawl-vectors.html


# Observations et environnement
---

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import importlib
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Paramètres globaux

In [3]:
random_state = 42

## Données d'entrainement

In [4]:
df_texts_recueil = pd.read_csv('data/demo/221123-TextInfox.csv')

In [70]:
df_infox = df_texts_recueil[df_texts_recueil['infox']==1.0].copy()

In [71]:
df_infox = df_infox.reset_index()

In [6]:
df_texts_afp =  pd.read_csv('data/tmp/221118-TextsAfp.csv')

In [7]:
df_texts_afp

,url,text,infox
0,https://www.francesoir.fr/politique-monde/pres...,Présidentielle au Brésil: Bolsonaro talonne Lu...,0.0
1,https://www.francesoir.fr/politique-france/la-...,La France lance un plan de sobriété énergétiqu...,0.0
2,https://www.francesoir.fr/politique-monde/mani...,Manifestations des femmes en Iran contre le po...,0.0
3,https://www.francesoir.fr/politique-france/aff...,Affaire Quatennens: LFI sous la pression de la...,0.0
4,https://www.francesoir.fr/politique-monde/covi...,"Covid-19: pour Joe Biden, ""la pandémie est ter...",0.0
...,...,...,...
3232,https://www.francesoir.fr/lifestyle-gastronomi...,La deuxième vie des coquilles d'huîtres\nLes h...,0.0
3233,https://www.francesoir.fr/lifestyle-gastronomi...,La truffe se fait rare\nVictime du réchauffeme...,0.0
3234,https://www.francesoir.fr/lifestyle-gastronomi...,Rungis prépare un Noël opulent malgré les atte...,0.0
3235,https://www.francesoir.fr/lifestyle-gastronomi...,Gastronomie: le Franco-Suisse Benoît Violier m...,0.0


In [8]:
df_words = pd.read_csv('data/prod/221125-InfoxWords.csv',index_col=0)

In [9]:
df_words

,count_0,count_1,count,rare_1,rare_2,rare_3,rare_4,rare_5,rare_6,rare_7,common_50,common_80,common_120,common_200,common_500
de,114412,23980,138392,False,False,False,False,False,False,False,True,True,True,True,True
la,63402,12901,76303,False,False,False,False,False,False,False,True,True,True,True,True
le,49361,9921,59282,False,False,False,False,False,False,False,True,True,True,True,True
et,38704,10142,48846,False,False,False,False,False,False,False,True,True,True,True,True
les,38053,10363,48416,False,False,False,False,False,False,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mayr,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False
mazari,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False
maçonnique,0,2,2,False,True,True,True,True,True,True,False,False,False,False,False
maïté,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False


In [10]:
ignore_words_small = list(df_words[df_words['common_80']].index)+list(df_words[df_words['rare_1']].index)
ignore_words_medium = list(df_words[df_words['common_80']].index)+list(df_words[df_words['rare_2']].index)
ignore_words_large = list(df_words[df_words['common_200']].index)+list(df_words[df_words['rare_5']].index)
ignore_words_xlarge = list(df_words[df_words['common_500']].index)+list(df_words[df_words['rare_7']].index)


# Experience
---

In [11]:
import spacy

In [26]:
#nlp = spacy.load("fr_core_news_sm")
nlp = spacy.load("fr_core_news_lg")
#nlp = spacy.load("fr_dep_news_trf")

In [64]:
9

9

In [76]:
text = df_infox['text'][1]
print(text)

"Un président doit savoir prendre ses responsabilités" - Lettre d'un médecin à E. Macron
LETTRE OUVERTE - Monsieur le Président,Je vais me faire vacciner contre le COVID. Cet acte médical se déroulera, de fait, en plein Été. Il a pour objectif de nous protéger, mes patients et moi, d’un coronavirus à ARN, extrêmement instable, à contaminations manu-portée et aérienne, et à variation saisonnière manifeste sous nos latitudes.En tant que médecin, l’ordre m’en est donc intimé. Dont act. Il n’est pas question pour moi de contrevenir à une injonction émanant du pouvoir politique de mon pays, formulée avec la bénédiction de mon autorité ordinale, et sur recommandations de l’Académie Nationale de Médecine. Encore moins question de produire de faux certificats, même si cette éventualité m’a été soufflée par de respectables confrères. Eux, ne manifestent ouvertement aucune opposition à cette campagne vaccinale, à laquelle ils se sont pourtant discrètement ainsi soustraits. Leur étrange mutisme t

In [77]:
doc = nlp(text)
tokens = []
for token in doc:
    item = {}
    item['text'] = token.text
    item['lemma_'] = token.lemma_
    item['pos_'] = token.pos_
    item['tag_'] = token.tag_
    item['dep_'] = token.dep_
    item['shape_'] = token.shape_
    item['is_alpha'] = token.is_alpha
    item['is_stop'] = token.is_stop
    tokens.append(item)
df_token = pd.DataFrame(tokens)
df_token

,text,lemma_,pos_,tag_,dep_,shape_,is_alpha,is_stop
0,"""","""",PUNCT,PUNCT,punct,"""",False,False
1,Un,un,DET,DET,det,Xx,True,True
2,président,président,NOUN,NOUN,nsubj,xxxx,True,False
3,doit,devoir,VERB,VERB,ROOT,xxxx,True,True
4,savoir,savoir,AUX,AUX,dep,xxxx,True,False
...,...,...,...,...,...,...,...,...
1429,",",",",PUNCT,PUNCT,punct,",",False,False
1430,mes,mon,DET,DET,det,xxx,True,True
1431,respectueuses,respectueux,ADJ,ADJ,amod,xxxx,True,False
1432,salutations,salutation,NOUN,NOUN,obj,xxxx,True,False


## NER

In [78]:
def get_ents(text):
    ents = {}
    doc = nlp(text)
    for ent in doc.ents:
        ents[ent.text] = ent.label_
#        print(ent.text)
    return ents


In [79]:
get_ents(text)

{'E. Macron\nLETTRE': 'PER',
 'Académie Nationale': 'ORG',
 'Conseil National de l’Ordre des Médecins': 'ORG',
 'Santé.': 'MISC',
 'Bordeaux': 'LOC',
 'Sud-Ouest': 'LOC',
 'Monsieur le Président': 'PER',
 'Delta': 'LOC',
 'COVID': 'MISC',
 'Europe': 'LOC',
 'Cov2': 'LOC',
 'SANOFI': 'ORG',
 'VALNEVA': 'LOC'}

## Feature Engineering

### Preprocessor

In [ ]:
import patat.util.text

importlib.reload(patat.util.text)
preprocessor = patat.util.text.preprocess
#preprocessor=None

### Tokenizer

In [ ]:
from patat.ml.lex_analyser import LexAnalyser

lex = LexAnalyser()

tokenizer = lex.get_lemmas_from_text

### TfIdfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#afp_size = 3000
#afp_size = 120
#afp_size = 50
afp_size = 20
#afp_size = 0
df_ml = pd.concat([df_texts_recueil,df_texts_afp.sample(afp_size,random_state=random_state)])

In [ ]:
%%time
#stop_words = ignore_words_small
#stop_words = ignore_words_medium
stop_words = ignore_words_large
#stop_words = ignore_words_xlarge
vectorizer = TfidfVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer, stop_words=stop_words)
#vectorizer = CountVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer, stop_words=stop_words)
count_matrix = vectorizer.fit_transform(df_ml['text'])
count_array = count_matrix.toarray()
df_tf = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())
df_tf.shape

df_X = df_tf
X = df_X

len(df_X.keys())

In [ ]:
df_tf

## Benchmark

### Initialisations

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import metrics
#from sklearn.metrics import roc_curve

In [ ]:
#X = df_X
y=df_ml['infox']

In [ ]:
df_X.shape

In [ ]:
train_size=0.8
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=random_state,train_size=train_size)

### Benchmark

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
predictors = {
    "Logistic Regression" : LogisticRegression(C=1000, random_state=random_state, solver='lbfgs', multi_class='ovr', max_iter=1000),
    "Neural Network" : MLPClassifier(alpha=1, max_iter=1000, hidden_layer_sizes=(8,8), random_state=random_state),
    "RBF SVM" : SVC(gamma=2, C=1, probability=True, random_state=random_state),
    "Gaussian Naive Bayes" : GaussianNB(),
    "Multinomial Naive Bayes" : MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None),
#    "Nearest Neighbors" : KNeighborsClassifier(3),
    "Linear SVM" : SVC(kernel="linear", C=0.025, probability=True, random_state=random_state),
    "Gaussian Process" : GaussianProcessClassifier(1.0 * RBF(1.0), random_state=random_state),
    "Decision Tree" : DecisionTreeClassifier(max_depth=5, random_state=random_state),
    "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=random_state),
    "AdaBoost" : AdaBoostClassifier(random_state=random_state),
    "QDA" : QuadraticDiscriminantAnalysis(),
    'Gradient Boosting': HistGradientBoostingClassifier(random_state=random_state),
    'Dummy' : DummyClassifier(strategy='uniform', random_state=random_state),
}


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=random_state,train_size=0.75)
y_preds = {}
y_probas = {}
fit_times = {}
proba_times = {}

#for name, classifier in zip(names, classifiers):
for name in predictors.keys():
    
    classifier = predictors[name]
    print(f'Training {name}')
    
    start = time.time()
    classifier.fit(X_train, y_train)
    stop = time.time()
    fit_times[name] = stop - start
    
    print(f'Training time : {fit_times[name]} s')
    
    y_preds[name] = classifier.predict(X_test)
    
    start = time.time()
    y_probas[name] = classifier.predict_proba(X_test)
    stop = time.time()
    proba_times[name] = stop - start
print('Done !')

## Mesure Résultats

### ROC et AUC

In [ ]:
plt.figure(0).clf()
for name in y_probas.keys():
    y_proba = y_probas[name][:, 1]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba)
    auc = round(metrics.roc_auc_score(y_test, y_proba), 4)
    plt.plot(fpr,tpr,label=f"{name}, AUC={auc}")
plt.legend()

In [ ]:
bench_results = {}
for key in y_preds.keys():
    bench_results[key] = {}
    fpr, tpr, _ = metrics.roc_curve(y_test, y_proba)
    bench_results[key]['roc_auc'] = metrics.roc_auc_score(y_test, y_probas[key][:, 1])
    bench_results[key]['accuracy']=metrics.accuracy_score(y_test, y_preds[key])
    bench_results[key]['recall']=metrics.recall_score(y_test, y_preds[key])
    bench_results[key]['f1']=metrics.f1_score(y_test, y_preds[key])
    bench_results[key]['log_loss'] = metrics.log_loss(y_test, y_preds[key])
    cnf_matrix = metrics.confusion_matrix(y_test, y_preds[key],normalize='all')
    bench_results[key]['false_positive']=cnf_matrix[0,1]
    bench_results[key]['false_negative']=cnf_matrix[1,0]
    bench_results[key]['fit_time']=fit_times[key]
    bench_results[key]['proba_time']=proba_times[key]
    

In [ ]:
df_bench = pd.DataFrame(bench_results).T
df_bench = df_bench.sort_values('roc_auc',ascending=False)

In [ ]:
print(f'Train size \t{len(y_train)} = {int(len(y_train)-y_train.sum())} False + {int(y_train.sum())} True' )
print(f'Test size \t{len(y_test)} = {int(len(y_test)-y_test.sum())} False + {int(y_test.sum())} True\n' )
pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
sns.scatterplot(df_bench,y='accuracy',x='roc_auc',hue=df_bench.index)

# Sauvegarde des résultats
---

# Conclusions
---
- Framework pour comparer les algorithmes : OK
- Résultats encourageants pour les prédictions.

- 

# Bricolages
---